In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import codecs
import collections
import pandas as pd
import numpy as np
import scipy as scp
import re
import os
from unicodedata import normalize

In [2]:
def remover_acentos_e_pontuacao(txt):
    sem_acentos = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    return re.sub('[^a-zA-Z0-9 ]', '', sem_acentos)
# print (remover_acentos_e_pontuacao("áàãâä! éèêë? íì&#297;îï, óòõôö; úù&#361;ûü"))


In [4]:
a = [1,2]
b = [4,5]
a +=b
a

[1, 2, 4, 5]

In [5]:
# função utilizada para baixar uma página html e savar em name
def getPage(url, name, codec):
    
    if not os.path.isfile(name):
        page = rq.get(url)
        f = codecs.open(name, 'w', codec)
        f.write(page.text)
        f.close

In [16]:
# utilizado para filtrar apenas of elementos html que são visiveis para o usuário
def tag_visible(element):
    
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def title_page(element):
    
    if element.parent.name in ['title']:
        return True
    return False

# utilizado para pegar apenas o texto da página html
def getText(arquivo):        
    bsFile = bs(open(arquivo),"html.parser")
    texts = bsFile.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    title = filter(title_page, texts)
    words = [t.strip() for t in visible_texts]
    words_title = [("title" + str(t.strip())) for t in title]
    words = filter(lambda x: len(x)>0, words)
    words_title = filter(lambda x: len(x)>0, words_title)
    words = list(map(lambda x: x.lower(), words))
    words_title = list(map(lambda x: x.lower(), words_title))
    words += words_title
    text = " ".join(words)
    return remover_acentos_e_pontuacao(text)

# utilizado para pegar as palavras que aparecem em uma página html (apenas páginas salvas localmente)
def getUniqueWords(arquivo):
    text = getText(arquivo)
    words = text.split(" ")
    words = filter(lambda word: len(word) > 0, words) # filtrar por palavras não vazias
    uniqueWords = collections.Counter(words)
    return uniqueWords

def getAllUniqueWords(diretorio):
    files = os.listdir(diretorio)
    files = filter(lambda x: x[-5:].lower() == ".html", files)
    allWords = []
    for index, f in enumerate(files):
        allWords = allWords + getUniqueWords(diretorio + "/" + f)
        print (index, len(allWords))
        
    uniqueWords = collections.Counter(allWords)
    return [word for word in uniqueWords.keys()]

In [10]:
# pega as palavras unicas dos sites
fr = codecs.open("palavrasAcentoTudo.txt", "w", "utf-8")
files = os.listdir("sites")
files = filter(lambda x: x[-5:].lower() == ".html", files)
words = []
for index, f in enumerate(files):
    w = getUniqueWords("sites/" + f)
    w = filter(lambda word: len(word) > 0, w)
    words += w
    print ("está no arquivo: " + str(f))
    
uwords = []
for w in words:
    if w not in uwords:
        uwords.append(w)
        
fr.write(",".join(uwords) + "\n")
fr.close()
    

está no arquivo: 31www.lojasrenner.com.br.html
está no arquivo: 148www.gregory.com.br.html
está no arquivo: 142www.gregory.com.br.html
está no arquivo: 137www.gregory.com.br.html
está no arquivo: 209www.dafiti.com.br.html
está no arquivo: 164www.zattini.com.br.html
está no arquivo: 27www.lojasrenner.com.br.html
está no arquivo: 155www.zattini.com.br.html
está no arquivo: 116www.cea.com.br.html
está no arquivo: 90www.marisa.com.br.html
está no arquivo: 102www.marisa.com.br.html
está no arquivo: 67mariadonata.com.br.html
está no arquivo: 42www.damyller.com.br.html
está no arquivo: 194produto.mercadolivre.com.br.html
está no arquivo: 201www.dafiti.com.br.html
está no arquivo: 208dafitimag.dafiti.com.br.html
está no arquivo: 169www.zattini.com.br.html
está no arquivo: 51www.damyller.com.br.html
está no arquivo: 145www.gregory.com.br.html
está no arquivo: 130www.gregory.com.br.html
está no arquivo: 126www.cea.com.br.html
está no arquivo: 163www.zattini.com.br.html
está no arquivo: 217www.da

está no arquivo: 86pesquisa.marisa.com.br.html
está no arquivo: 172www.zattini.com.br.html
está no arquivo: 204www.dafiti.com.br.html
está no arquivo: 114www.cea.com.br.html
está no arquivo: 119www.cea.com.br.html
está no arquivo: 71mariadonata.com.br.html
está no arquivo: 53www.damyller.com.br.html
está no arquivo: 179loja.mercadolivre.com.br.html
está no arquivo: 198produto.mercadolivre.com.br.html
está no arquivo: 171www.zattini.com.br.html
está no arquivo: 97www.marisa.com.br.html
está no arquivo: 105www.marisa.com.br.html
está no arquivo: 117www.cea.com.br.html
está no arquivo: 82mariadonata.com.br.html
está no arquivo: 188roupas.mercadolivre.com.br.html
está no arquivo: 222www.dafiti.com.br.html
está no arquivo: 44www.damyller.com.br.html
está no arquivo: 18www.riachuelo.com.br.html
está no arquivo: 72mariadonata.com.br.html
está no arquivo: 178www.mercadolivre.com.br.html
está no arquivo: 78mariadonata.com.br.html
está no arquivo: 218www.dafiti.com.br.html
está no arquivo: 150ww

In [18]:
fr = codecs.open("palavrasAcentoSelecionadas.txt", "r", "utf-8")
uwordsSelected = fr.read().split(",")
fr.close()

In [12]:
# utilizado para ler o arquivo com as páginas utilizadas no projeto
data_set = pd.read_csv("sites.csv")

In [94]:
# utizado para baixar as páginas html que foram utilizadas no projeto

padrao = '((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)'
for i, site in enumerate(data_set.url):
    name = str(i) + re.findall(padrao, site)[0][0] + ".html"
    getPage(url=site, name="sites/" + name, codec="utf-8")
    print "finalizou " + name

In [19]:
# utilizado para pegar as repetições de cada palavra em cada página
fr = codecs.open("baseAcentoSelecionados.csv", "w", "utf-8")
fr.write(",".join(uwordsSelected) + ",classePagina\n")
files = os.listdir("sites")
files = filter(lambda x: x[-5:].lower() == ".html", files)
for index, f in enumerate(files):
# if True:
    allWords = getUniqueWords("sites" + "/" + f)

#     palavras = collections.Counter(allWords) # contar quantidade de cada palavras
    print (allWords)

    inputSite = []
    for pp in uwordsSelected:
        inputSite.append(allWords[pp])
    
    inputSite = ",".join(map(str, inputSite))
    
    padrao = '([0-9]+)'
    fr.write(inputSite + "," + data_set.anuncio[int(re.findall(padrao, f)[0])] + "\n")
    print ("está no arquivo: " + str(f) + " --> " + data_set.anuncio[int(re.findall(padrao, f)[0])])

fr.close()

Counter({'e': 35, 'de': 31, 'a': 19, 'para': 18, 'frete': 13, 'r': 11, 'renner': 11, 'o': 10, 'veja': 9, 'cartao': 9, 'gratis': 9, 'feminino': 8, 'calcados': 8, 'email': 7, 'senha': 7, 'com': 7, 'masculino': 7, 'seu': 7, 'infantil': 7, 'mais': 7, 'ofertas': 7, 'em': 7, 'compras': 7, 'do': 7, 'ou': 6, 'moda': 6, 'jeans': 6, 'tamanho': 6, 'satinato': 6, 'ate': 6, 'novidades': 5, 'por': 5, 'as': 5, 'curve': 5, 'plus': 5, 'size': 5, '1': 5, 'botas': 5, 'partir': 5, 'nas': 5, 'sua': 5, 'cadastrar': 4, 'ashua': 4, 'lojas': 4, 'intima': 4, 'relogios': 4, 'oculos': 4, 'perfumaria': 4, 'cosmeticos': 4, 'tendencias': 4, 'marcas': 4, 'refinementnamerefinementsplit': 4, 'feminina': 4, 'verniz': 4, 'no': 4, 'privacidade': 4, 'uso': 4, 'politicas': 4, 'termos': 4, 'cpf': 3, 'pagamento': 3, 'nossas': 3, 'bolsas': 3, 'total': 3, 'street': 3, 'style': 3, 'bermudas': 3, 'camisetas': 3, 'calcas': 3, 'casacos': 3, 'jaquetas': 3, 'acessorios': 3, 'praia': 3, 'tenis': 3, 'chinelos': 3, 'off': 3, 'bota': 3, 

Counter({'r': 139, 'de': 98, 'bermuda': 57, 'e': 52, 'favoritos': 49, 'espiar': 48, '0': 45, 'mais': 43, 'ver': 35, 'jeans': 31, 'promocoes': 23, 'novidades': 23, 'para': 22, 'broken': 21, 'rules': 21, '2x': 20, 'polo': 19, 'a': 17, 'tenis': 17, 'wear': 17, 'azul': 17, '3x': 16, 'adidas': 15, 'dafiti': 13, 'clothing': 13, 'colcci': 11, 'nike': 11, 'sarja': 11, '9990': 11, 'o': 10, 'com': 10, 'roupas': 9, 'forum': 9, 'acessorios': 8, 'bermudas': 8, 'calvin': 8, 'klein': 8, 'brasil': 8, 'fatal': 8, 'surf': 8, 'gladiadores': 8, 'masculina': 8, 'do': 7, 'moletom': 7, 'em': 7, 'co': 7, 'performance': 7, '1x': 7, 'as': 6, 'calcados': 6, 'calca': 6, 'top': 6, 'marcas': 6, 'fiveblu': 6, 'que': 6, 'reserva': 6, 'vestido': 6, 'os': 6, 'na': 6, 'limpar': 6, 'store': 6, 'alkary': 6, 'fit': 6, 'blue': 6, 'moda': 6, '3330': 6, 'reta': 6, 'informacoes': 5, 'feminino': 5, 'vestidos': 5, 'esportiva': 5, 'santa': 5, 'shoes': 5, 'ellus': 5, 'masculino': 5, 'lacoste': 5, 'asics': 5, 'fitness': 5, 'puma': 

Counter({'e': 35, 'de': 31, 'a': 19, 'para': 18, 'frete': 13, 'r': 11, 'renner': 11, 'com': 10, 'o': 10, 'veja': 9, 'cartao': 9, 'gratis': 9, 'feminino': 8, 'calcados': 8, 'do': 8, 'email': 7, 'senha': 7, 'masculino': 7, 'seu': 7, 'infantil': 7, 'mais': 7, 'ofertas': 7, 'compras': 7, 'ou': 6, 'moda': 6, 'jeans': 6, 'tamanho': 6, 'satinato': 6, 'ate': 6, 'novidades': 5, 'por': 5, 'as': 5, 'curve': 5, 'plus': 5, 'size': 5, '1': 5, 'botas': 5, 'partir': 5, 'nas': 5, 'sua': 5, 'cadastrar': 4, 'ashua': 4, 'lojas': 4, 'intima': 4, 'relogios': 4, 'oculos': 4, 'perfumaria': 4, 'cosmeticos': 4, 'tendencias': 4, 'marcas': 4, 'refinementnamerefinementsplit': 4, 'feminina': 4, 'ziper': 4, 'em': 4, 'no': 4, 'privacidade': 4, 'uso': 4, 'politicas': 4, 'termos': 4, 'cpf': 3, 'pagamento': 3, 'nossas': 3, 'bolsas': 3, 'total': 3, 'street': 3, 'style': 3, 'bermudas': 3, 'camisetas': 3, 'calcas': 3, 'casacos': 3, 'jaquetas': 3, 'acessorios': 3, 'praia': 3, 'tenis': 3, 'chinelos': 3, 'off': 3, 'bota': 3, 

Counter({'calca': 82, 'feminina': 79, 'marisa': 70, 'de': 44, 'jeans': 44, 'r': 38, 'detalhes': 32, 'espiar': 32, 'e': 28, '2': 25, 'a': 22, 'calcados': 22, 'calcas': 19, 'skinny': 19, 'legging': 17, 'infantil': 16, '1': 16, 'moda': 15, 'fitness': 15, '3': 15, 'flare': 14, 'ofertas': 13, 'cigarrete': 13, 'plus': 12, 'size': 12, 'veludo': 12, 'novidades': 11, 'masculino': 11, 'lingerie': 11, 'acessorios': 11, 'anos': 11, 'feminino': 10, 'mais': 10, 'sawary': 10, 'tenis': 9, 'destroyed': 9, 'com': 9, 'roupas': 8, 'social': 8, 'marcas': 8, 'conosco': 7, 'camisas': 7, 'saias': 7, 'shorts': 7, 'por': 7, '4': 7, 'meias': 7, 'preco': 7, 'mulher': 7, 'lojas': 6, '0': 6, 'na': 6, 'moletom': 6, 'vestidos': 6, 'setores': 6, 'praia': 6, 'menina': 6, 'menino': 6, 'sarja': 6, 'relogios': 6, '10': 6, '5': 6, 'gups': 6, 'como': 5, 'femininas': 5, 'bermudas': 5, 'cartao': 5, 'tendencias': 5, 'floral': 5, 'bota': 5, 'chinelo': 5, 'via': 5, 'sandalias': 5, 'meninas': 5, 'meninos': 5, 'ver': 5, 'tudo': 5,

Counter({'facebook': 7, 'do': 6, 'o': 6, 'wifi': 6, 'no': 5, 'codigo': 5, 'para': 4, 'ou': 4, 'cadastrese': 4, 'e': 4, 'em': 4, 'vez': 4, 'disso': 4, 'de': 3, 'a': 3, 'checkin': 3, 'use': 3, 'entrar': 2, 'ufpe': 2, 'faculdade': 2, 'universidade': 2, 'faca': 2, 'acesso': 2, 'internet': 2, 'clique': 2, 'link': 2, 'abaixo': 2, 'pular': 1, 'secoes': 1, 'desta': 1, 'pagina': 1, 'ajuda': 1, 'acessibilidade': 1, 'pressione': 1, 'alt': 1, 'abrir': 1, 'este': 1, 'menu': 1, 'participe': 1, 'entre': 1, 'email': 1, 'telefone': 1, 'senha': 1, 'esqueceu': 1, 'conta': 1, 'quer': 1, 'participar': 1, 'cin': 1, 'escola': 1, 'obter': 1, 'gratis': 1, 'inserir': 1, 'gratuito': 1, 'fazer': 1, 'hoje': 1, 'ao': 1, 'usar': 1, 'um': 1, 'voce': 1, 'entende': 1, 'aceita': 1, 'os': 1, 'termos': 1, 'volte': 1, '2018': 1, 'titlecin': 1, 'com': 1})
está no arquivo: 67mariadonata.com.br.html --> não
Counter({'r': 182, 'de': 125, 'comprar': 105, 'em': 101, 'ate': 94, 'ou': 94, 'por': 93, '0': 91, 'espiar': 91, '5x': 74

...

Counter({'r': 88, 'para': 52, 'o': 50, 'todo': 48, 'pais': 48, 'vendidos': 48, 'envio': 43, 'roupas': 39, 'sao': 39, 'paulo': 39, 'femininas': 32, 'cores': 31, 'sem': 26, 'juros': 26, 'feminina': 24, '6': 22, '12x': 19, 'roupa': 17, '5': 16, '99': 16, 'moda': 15, '4': 14, 'e': 13, '90': 12, 'mercado': 11, 'de': 11, 'rio': 11, '3': 10, 'blusas': 10, 'do': 10, 'gratis': 10, '2': 10, 'atacado': 9, 'frete': 9, 'vestido': 9, '8': 8, 'livre': 7, 'feminino': 7, 'com': 7, '10': 7, '6x': 7, 'kit': 7, 'em': 6, 'mais': 6, 'grande': 6, 'sul': 6, 'conjunto': 6, 'short': 6, 'calca': 6, 'renda': 6, '7': 6, 'blazer': 6, 'vender': 5, 'a': 5, 'vestidos': 5, 'janeiro': 5, 'santa': 5, 'catarina': 5, '24': 5, '15': 5, '33': 5, 'cropped': 5, '44': 5, 'blusa': 5, '19': 5, '3x': 5, '83': 5, '20': 5, '49': 5, 'bolsas': 4, 'jeans': 4, 'preco': 4, 'fitness': 4, 'distrito': 4, 'federal': 4, '9': 4, '17': 4, 'festa': 4, '29': 4, 'blogueiras': 4, 'revenda': 4, '50': 4, 'tshirts': 4, '69': 4, 'colete': 4, 'peplum': 

Counter({'e': 35, 'de': 28, 'a': 20, 'para': 19, 'frete': 12, 'o': 10, 'r': 9, 'gratis': 9, 'renner': 9, 'veja': 8, 'cartao': 8, 'ou': 7, 'senha': 7, 'feminino': 7, 'masculino': 7, 'infantil': 7, 'calcados': 7, 'mais': 7, 'ofertas': 7, 'compras': 7, 'do': 7, 'email': 6, 'com': 6, 'moda': 6, 'jeans': 6, 'ate': 6, 'novidades': 5, 'por': 5, 'seu': 5, 'as': 5, 'curve': 5, 'plus': 5, 'size': 5, 'tamanho': 5, 'partir': 5, 'nas': 5, 'sua': 5, 'cadastrar': 4, 'ashua': 4, 'lojas': 4, 'intima': 4, 'relogios': 4, 'oculos': 4, 'perfumaria': 4, 'cosmeticos': 4, 'tendencias': 4, 'marcas': 4, '1': 4, 'refinementnamerefinementsplit': 4, 'esta': 4, 'privacidade': 4, 'uso': 4, 'politicas': 4, 'termos': 4, 'cpf': 3, 'nossas': 3, 'bolsas': 3, 'total': 3, 'street': 3, 'style': 3, 'bermudas': 3, 'camisetas': 3, 'calcas': 3, 'casacos': 3, 'jaquetas': 3, 'acessorios': 3, 'praia': 3, 'tenis': 3, 'chinelos': 3, 'botas': 3, 'off': 3, 'que': 3, 'voce': 3, 'valor': 3, 'da': 3, 'no': 3, 'compra': 3, 'meu': 3, 'em':

Counter({'e': 40, 'de': 35, 'a': 24, 'para': 19, 'frete': 13, 'infantil': 12, 'r': 12, 'renner': 11, 'o': 10, 'veja': 9, 'do': 9, 'cartao': 9, 'gratis': 9, 'com': 8, 'moda': 8, 'mais': 8, 'email': 7, 'senha': 7, 'feminino': 7, 'masculino': 7, 'seu': 7, 'calcados': 7, 'ofertas': 7, 'compras': 7, 'ou': 6, '6': 6, 'por': 6, 'as': 6, 'jeans': 6, 'camisetas': 6, '14': 6, 'ate': 6, 'em': 6, 'novidades': 5, 'curve': 5, 'plus': 5, 'size': 5, 'tendencias': 5, 'tamanho': 5, '1': 5, 'partir': 5, 'nas': 5, 'sua': 5, 'cadastrar': 4, 'ashua': 4, 'lojas': 4, 'intima': 4, 'relogios': 4, 'oculos': 4, 'perfumaria': 4, 'cosmeticos': 4, 'bermudas': 4, 'marcas': 4, 'fuzarka': 4, 'off': 4, 'refinementnamerefinementsplit': 4, 'listrada': 4, 'que': 4, 'da': 4, 'no': 4, 'privacidade': 4, 'uso': 4, 'politicas': 4, 'termos': 4, 'cpf': 3, 'pagamento': 3, 'nossas': 3, 'bolsas': 3, 'total': 3, 'street': 3, 'style': 3, 'calcas': 3, 'casacos': 3, 'jaquetas': 3, 'acessorios': 3, 'praia': 3, 'linha': 3, 'menino': 3, 't

Counter({'e': 91, 'de': 26, 'para': 25, 'calca': 14, 'a': 12, 'acessorios': 12, 'camiseta': 11, 'personagens': 11, 'calcados': 11, 'jaqueta': 10, 'moletom': 10, 'riachuelo': 9, 'short': 9, 'bermuda': 9, 'casaco': 8, 'sueter': 8, 'pijama': 8, 'em': 6, 'macacao': 6, 'saia': 6, 'meias': 6, 'meninos': 6, 'r': 5, 'colecao': 5, 'camisa': 5, 'regata': 5, 'pool': 5, 'meninas': 5, 'tenis': 5, 'luva': 5, 'toalha': 5, 'pagina': 5, 'entrar': 4, 'feminino': 4, 'vestido': 4, 'body': 4, 'moda': 4, 'relogios': 4, 'perfumes': 4, 'bota': 4, 'chinelo': 4, 'mocassim': 4, 'sandalia': 4, 'bone': 4, 'chapeu': 4, 'cachecol': 4, 'gorro': 4, 'mochila': 4, 'cama': 4, 'jogo': 4, 'banho': 4, 'cartoes': 4, 'ir': 3, 'o': 3, 'conta': 3, 'sua': 3, 'lojas': 3, 'macaquinho': 3, 'blusa': 3, 'praia': 3, 'destaque': 3, 'beleza': 3, 'jeans': 3, 'ak': 3, 'casual': 3, 'masculino': 3, 'polo': 3, 'cueca': 3, 'infantil': 3, 'garotas': 3, 'garotos': 3, 'cinto': 3, 'diversos': 3, 'rosto': 3, 'linha': 3, 'manta': 3, 'tapete': 3, 'c

Counter({'e': 48, 'de': 44, 'a': 27, 'para': 26, 'renner': 26, 'cartao': 20, 'o': 16, 'no': 16, 'com': 15, 'em': 13, 'brinde': 12, 'x': 12, 's': 12, 'juros': 12, 'frete': 12, 'as': 11, 'mais': 11, 'jeans': 10, 'r': 10, 'moda': 9, 'gratis': 9, 'por': 8, 'veja': 8, 'senha': 7, 'feminino': 7, 'masculino': 7, 'seu': 7, 'infantil': 7, 'calcados': 7, 'ofertas': 7, 'ate': 7, 'voce': 7, 'da': 7, 'compras': 7, 'do': 7, 'email': 6, 'ou': 6, 'novidades': 6, 'lojas': 6, 'nas': 6, 'curve': 5, 'plus': 5, 'size': 5, 'camisetas': 5, 'calcas': 5, 'tamanho': 5, 'partir': 5, 'que': 5, 'todas': 5, 'sua': 5, 'cadastrar': 4, 'ashua': 4, 'nossas': 4, 'intima': 4, 'relogios': 4, 'oculos': 4, 'perfumaria': 4, 'cosmeticos': 4, 'tendencias': 4, 'veludo': 4, 'blusas': 4, 'vestidos': 4, 'marcas': 4, '1': 4, 'refinementnamerefinementsplit': 4, 'femininas': 4, 'sao': 4, 'site': 4, 'os': 4, 'maior': 4, 'privacidade': 4, 'uso': 4, 'politicas': 4, 'termos': 4, 'cpf': 3, 'nome': 3, 'pagamento': 3, 'bolsas': 3, 'tudo': 3

In [35]:
fr = codecs.open("teste.arff", "w", "utf-8")
for u in uwordsSelected:
    fr.write("@ATTRIBUTE " + u + " NUMERIC\n")

fr.write("@ATTRIBUTE classePagina {sim, não}\n")
fr.write("\n\n@DATA\n")
fr.close()